In [3]:
!pip install streamlit
!pip install streamlit ngrok
!pip install yfinance


     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     -------------------------------------- - 2.9/3.0 MB 16.8 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 16.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for peewee: filename=peewee-3.17.9-py3-none-any.whl size=139127 sha256=8c1a0cd321bb0094b89df182b82c0b73a6eda7d2b9d60dda6d8e5a01ecda437c
  Stored in directory: c:\users\arunesh\appdata\local\pip\cache\wheels\43\ef\2d\2c51d496bf084945ffdf838b4cc8767b8ba1cc20eb41588831
Successfully built peewee


### Fetching a live stock data

In [4]:
import yfinance as yf
import pandas as pd
import plotly.express as px

# Define top 50 tech stock tickers
top_tech_tickers = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "NVDA", "META", "ADBE", "NFLX", "CRM",
    "AMD", "INTC", "ORCL", "IBM", "CSCO", "UBER", "PYPL", "SHOP", "SQ", "SNOW",
    "ASML", "AVGO", "TXN", "QCOM", "NOW", "ZM", "TWLO", "PLTR", "DOCU", "WDAY",
    "TEAM", "FSLY", "CRWD", "OKTA", "DDOG", "NET", "PINS", "SNAP", "ROKU", "SPOT",
    "ZS", "MDB", "VEEV", "PANW", "FIS", "ADSK", "SAP", "INTU", "ANET", "SE"
]

# Function to get live stock data and market cap
def get_stock_data(tickers):
    data = []
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            hist = stock.history(period="1d", interval="5m")  # 5-minute interval

            if hist.empty:
                print(f"Warning: No recent data available for {ticker}, skipping...")
                continue

            latest = hist.iloc[-1]  # Get the most recent data

            data.append({
                "Ticker": ticker,
                "Price": latest["Close"],
                "Change (%)": ((latest["Close"] - latest["Open"]) / latest["Open"]) * 100,
                "Market Cap": info.get("marketCap", 0)  # Get market cap (0 if unavailable)
            })
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
    return pd.DataFrame(data)

# Fetch stock data
df = get_stock_data(top_tech_tickers)

# Convert Market Cap to Billions
df["Market Cap (B)"] = df["Market Cap"] / 1e9  # Convert to billions

# Display DataFrame
print(df)


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SQ?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SQ&crumb=Re%2Fy.7pTj.g


Error fetching data for SQ: 'NoneType' object has no attribute 'update'
   Ticker       Price  Change (%)     Market Cap  Market Cap (B)
0    AAPL  172.770004    0.063709  2590110646272     2590.110646
1    MSFT  355.570007    0.138003  2635791859712     2635.791860
2   GOOGL  144.699997   -0.027642  1777393467392     1777.393467
3    AMZN  171.089996    0.134611  1808603414528     1808.603415
4    TSLA  221.580002   -0.267807   713617113088      713.617113
5    NVDA   96.709999    0.203490  2349719879680     2349.719880
6    META  510.790009    0.282714  1293306691584     1293.306692
7    ADBE  340.019989    0.366016   144907993088      144.907993
8    NFLX  870.750000    0.272921   372319682560      372.319683
9     CRM  244.039993    0.205305   234474389504      234.474390
10    AMD   78.190002    0.012794   127062310912      127.062311
11   INTC   18.150000    0.693481    79057485824       79.057486
12   ORCL  124.459999    0.298173   349126623232      349.126623
13    IBM  221.050

### Stock Real-Time Heatmap

In [5]:

# Create a Heatmap with Market Cap as Size
fig = px.treemap(df,
                 path=["Ticker"],
                 values="Market Cap (B)",  # Market Cap as size
                 color="Change (%)",  # Color based on percentage change
                 color_continuous_scale=["red", "white", "green"],
                 title="Live Tech Stock Heatmap (Sized by Market Cap)")

fig.show()


### Install Libraries

In [6]:
!pip install yfinance ipywidgets

### Investment Calculator

In [7]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Define top 50 tech stocks
top_tech_stocks = {
    "Apple (AAPL)": "AAPL", "Microsoft (MSFT)": "MSFT", "Google (GOOGL)": "GOOGL", "Amazon (AMZN)": "AMZN",
    "Tesla (TSLA)": "TSLA", "Nvidia (NVDA)": "NVDA", "Meta (META)": "META", "Adobe (ADBE)": "ADBE",
    "Netflix (NFLX)": "NFLX", "Salesforce (CRM)": "CRM", "AMD": "AMD", "Intel (INTC)": "INTC",
    "Oracle (ORCL)": "ORCL", "IBM": "IBM", "Cisco (CSCO)": "CSCO", "Uber (UBER)": "UBER",
    "Paypal (PYPL)": "PYPL", "Shopify (SHOP)": "SHOP", "Square (SQ)": "SQ", "Snowflake (SNOW)": "SNOW",
    "ASML": "ASML", "Broadcom (AVGO)": "AVGO", "Texas Instruments (TXN)": "TXN", "Qualcomm (QCOM)": "QCOM",
    "ServiceNow (NOW)": "NOW", "Zoom (ZM)": "ZM", "Twilio (TWLO)": "TWLO", "Palantir (PLTR)": "PLTR",
    "DocuSign (DOCU)": "DOCU", "Workday (WDAY)": "WDAY", "Atlassian (TEAM)": "TEAM", "Fastly (FSLY)": "FSLY",
    "CrowdStrike (CRWD)": "CRWD", "Okta (OKTA)": "OKTA", "Datadog (DDOG)": "DDOG", "Cloudflare (NET)": "NET",
    "Pinterest (PINS)": "PINS", "Snap (SNAP)": "SNAP", "Roku (ROKU)": "ROKU", "Spotify (SPOT)": "SPOT",
    "Zscaler (ZS)": "ZS", "MongoDB (MDB)": "MDB", "Veeva (VEEV)": "VEEV", "Palo Alto Networks (PANW)": "PANW",
    "Fiserv (FIS)": "FIS", "Autodesk (ADSK)": "ADSK", "SAP": "SAP", "Intuit (INTU)": "INTU",
    "Arista Networks (ANET)": "ANET", "Sea Limited (SE)": "SE"
}

# Function to calculate compound returns
def calculate_returns(amount, years, annual_return):
    return amount * (1 + annual_return / 100) ** years

# Function to fetch stock's past 5-year CAGR
def get_cagr(stock_ticker):
    try:
        stock = yf.Ticker(stock_ticker)
        hist = stock.history(period="5y")  # Fetch last 5 years of data

        if hist.empty:
            return 10  # Default CAGR if data is missing

        start_price = hist["Close"].iloc[0]
        end_price = hist["Close"].iloc[-1]

        # Calculate CAGR: ((End Price / Start Price)^(1/years)) - 1
        cagr = ((end_price / start_price) ** (1 / 5) - 1) * 100
        return round(cagr, 2)
    except:
        return 10  # Default to 10% CAGR if fetch fails

# Create widgets for selecting multiple stocks
stock_dropdown_1 = widgets.Dropdown(options=top_tech_stocks.keys(), description="Stock 1:")
stock_dropdown_2 = widgets.Dropdown(options=top_tech_stocks.keys(), description="Stock 2:")
stock_dropdown_3 = widgets.Dropdown(options=top_tech_stocks.keys(), description="Stock 3:")

# Investment Inputs
amount_input_1 = widgets.IntText(value=1000, min=100, step=100, description="Investment ($):")
years_slider_1 = widgets.IntSlider(value=5, min=1, max=30, step=1, description="Years:")

amount_input_2 = widgets.IntText(value=1000, min=100, step=100, description="Investment ($):")
years_slider_2 = widgets.IntSlider(value=5, min=1, max=30, step=1, description="Years:")

amount_input_3 = widgets.IntText(value=1000, min=100, step=100, description="Investment ($):")
years_slider_3 = widgets.IntSlider(value=5, min=1, max=30, step=1, description="Years:")

# Button to calculate
calc_button = widgets.Button(description="Compare Investments", button_style='success')

# Output widget
output = widgets.Output()

# Function to update investment growth plots
def update_investment(change):
    with output:
        output.clear_output()

        fig, ax = plt.subplots(figsize=(8, 5))

        for i, (stock_dropdown, amount_input, years_slider) in enumerate([
            (stock_dropdown_1, amount_input_1, years_slider_1),
            (stock_dropdown_2, amount_input_2, years_slider_2),
            (stock_dropdown_3, amount_input_3, years_slider_3),
        ]):
            # Get user input
            selected_stock = stock_dropdown.value
            selected_ticker = top_tech_stocks[selected_stock]
            amount = amount_input.value
            years = years_slider.value

            # Fetch CAGR
            expected_return = get_cagr(selected_ticker)

            # Calculate projected return
            final_value = calculate_returns(amount, years, expected_return)

            # Display results
            print(f"📊 **Stock {i+1}:** {selected_stock} ({selected_ticker})")
            print(f"📈 **Historical Growth Rate:** {expected_return}% CAGR")
            print(f"💰 **Investment:** ${amount:,} for {years} years")
            print(f"💸 **Projected Return:** **${final_value:,.2f}**\n")

            # Investment Growth Chart
            x = np.arange(0, years + 1)
            y = [calculate_returns(amount, i, expected_return) for i in x]

            ax.plot(x, y, marker="o", linestyle="-", label=f"{selected_stock}")

        ax.set_title("Investment Growth Comparison")
        ax.set_xlabel("Years")
        ax.set_ylabel("Total Value ($)")
        ax.legend()
        ax.grid(True)
        ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'${x:,.0f}'))

        plt.show()

# Attach function to button click
calc_button.on_click(update_investment)

# Display UI
display(stock_dropdown_1, amount_input_1, years_slider_1,
        stock_dropdown_2, amount_input_2, years_slider_2,
        stock_dropdown_3, amount_input_3, years_slider_3,
        calc_button, output)


Dropdown(description='Stock 1:', options=('Apple (AAPL)', 'Microsoft (MSFT)', 'Google (GOOGL)', 'Amazon (AMZN)…

IntText(value=1000, description='Investment ($):', step=100)

IntSlider(value=5, description='Years:', max=30, min=1)

Dropdown(description='Stock 2:', options=('Apple (AAPL)', 'Microsoft (MSFT)', 'Google (GOOGL)', 'Amazon (AMZN)…

IntText(value=1000, description='Investment ($):', step=100)

...

Dropdown(description='Stock 3:', options=('Apple (AAPL)', 'Microsoft (MSFT)', 'Google (GOOGL)', 'Amazon (AMZN)…

IntText(value=1000, description='Investment ($):', step=100)

IntSlider(value=5, description='Years:', max=30, min=1)

Button(button_style='success', description='Compare Investments', style=ButtonStyle())

Output()